<a href="https://colab.research.google.com/github/girijesh-ai/opensource-llm/blob/main/Girijesh_2_hugging_face_pipelines_under_the_hood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial, you will perform three NLP tasks using Hugging Face tokenizers, models, and pipelines. The goal is to learn:

- How to use Hugging Face tokenizers for preprocessing tasks like padding, truncation, and batching of text.

- How model configuration works, including mapping between id2label and label2id for token classification tasks.

- How Hugging Face models work, including passing text through the model to generate logits.

- How to use the logits output from Hugging Face models to make predictions for your NLP task.

- How to recreate Hugging Face pipelines by using the tokenizers and models directly, instead of relying on the pipelines.

- Compare the results of using the tokenizers and models directly versus using the Hugging Face pipelines to evaluate the differences.

The focus of this tutorial is gaining hands-on experience with Hugging Face tokenizers, configuration, models, and pipelines through implementing three text processing tasks end-to-end. This will provide a deeper understanding of how these key NLP components work.

# Installing Core NLP Libraries

This section installs 3 key libraries for NLP and ML projects:

- Transformers - Provides access to pretrained models like BERT, RoBERTa for NLP.

- Datasets - Provides convenient access to common NLP datasets.

- Rich - For nicely formatted console output when training models.

Installing these libraries in one line allows quick setup of the Python environment with critical functionality for working on text data.


In [1]:
!pip install transformers datasets rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00


The transformers.pipeline() method provides quick access to pretrained NLP models for making predictions. The rich.pretty.pprint() method prints Python objects to the console in a readable formatted way.

In [2]:
from transformers import pipeline
from rich.pretty import pprint
import torch.nn.functional as F
import torch

Here is documentation for each section of the notebook:




# Summary

By walking through the pipeline components, this shows how to go from raw text to formatted predictions step-by-step. This provides more visibility than just using the packaged pipeline.

# Creating a Text Classification Pipeline

This section creates a text classification pipeline using Hugging Face's transformers library. The pipeline gives quick access to a pretrained DistilBERT model finetuned on the SST-2 sentiment analysis dataset.

The pipeline makes predictions on some sample text, returning the sentiment label and score for each sentence.

In [3]:
classification = pipeline(task="text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
results = classification(raw_inputs)
pprint(results)

[{'label': 'POSITIVE', 'score': 0.9598050713539124}, {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

# Loading Tokenizer, Config, and Model

This section loads the lower-level components used by the pipeline:

- Tokenizer: Preprocesses the text into ids, handles padding/truncation.

- Config: Contains model configuration like hyperparams and mapping from ids to labels.

- Model: The core Transformer model like DistilBERT that generates embeddings and predictions.

Loading these separately gives more control than just using the packaged pipeline.


In [5]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

In [6]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


# Tokenizing the Text

The tokenizer is used to preprocess the raw text into tokenized ids with padding & truncation to fit the expected model input shape.

This shows how the tokenizer prepares the data before passing it to the model.


In [8]:
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
pprint(inputs)

{
│   'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
│   │     2607,  2026,  2878,  2166,  1012,   102],
│   │   [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
│   │   │    0,     0,     0,     0,     0,     0]]),
│   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
│   │   [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])
}

# Printing Truncated Text

The truncated input text is decoded back to readable text using the tokenizer's decode method.

This shows how padding and truncation end up masking part of the original input.

# Passing Inputs to Model

The tokenized & padded inputs are passed to the model to generate predictions.

This uses the model directly instead of the pipeline, giving more control.


In [9]:
pprint(tokenizer.decode(inputs["input_ids"][1]))

'[CLS] i hate this so much! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

# Interpreting Model Outputs

The raw numeric tensor outputs of the model are converted into probability scores and sentiment labels.

This uses the mapping in the config to go from indices predicted by the model back to the associated labels.

The end result matches what the pipeline originally produced.


In [10]:
outputs = model(**inputs)
pprint(outputs)

SequenceClassifierOutput(
│   loss=None,
│   logits=tensor([[-1.5607,  1.6123],
│   │   [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>),
│   hidden_states=None,
│   attentions=None
)

## Covert logits to probabilities

In [11]:
predictions = F.softmax(outputs.logits, dim=-1)
pprint(predictions)

tensor([[4.0195e-02, 9.5981e-01],
│   │   [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)

## Loop through probabilities and convert to interpretable results

In [12]:
result = []
for index, prediction in enumerate(predictions):
  probability = torch.max(prediction).item()
  sentiment = config.id2label[torch.argmax(prediction).item()]
  result.append({"probability": probability, "label": sentiment})
pprint(result)

[
│   {'probability': 0.9598050713539124, 'label': 'POSITIVE'},
│   {'probability': 0.9994558691978455, 'label': 'NEGATIVE'}
]

# Creating a Token Classification Pipeline

This section creates a named entity recognition (NER) pipeline using the Hugging Face transformers library. The pipeline provides quick access to a pretrained BERT model finetuned on the CoNLL 2003 NER dataset.

The pipeline makes predictions on a sample input text, returning the predicted NER tags with scores for each token.


In [13]:
token_classifier = pipeline("token-classification")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Inspecting the Pipeline Output

The raw JSON output from the NER pipeline is printed to inspect the predicted entity, score, index, word, start and end values for each tagged token.

In [14]:
ner_raw_inputs = "My name is Wolfgang and I live in Berlin"
result = token_classifier(ner_raw_inputs)
pprint(result)

[
│   {'entity': 'I-PER', 'score': 0.99914825, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19},
│   {'entity': 'I-LOC', 'score': 0.9983669, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}
]

# Loading the Pipeline Components

The lower level tokenizer, config, and model objects that compose the pipeline are loaded. This gives more control than just using the packaged pipeline.


In [15]:
ner_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
ner_tokenizer = AutoTokenizer.from_pretrained(ner_checkpoint)
pprint(ner_tokenizer)

BertTokenizerFast(name_or_path='dbmdz/bert-large-cased-finetuned-conll03-english', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
│   │   0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
│   │   100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
│   │   101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
│   │   102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
│   │   103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

# Tokenizing the Input

The tokenizer preprocesses the raw text into tokenized ids, padding & truncating as needed to match the expected model input shape.


In [16]:
ner_model_inputs = ner_tokenizer(ner_raw_inputs, padding=True, truncation=True, return_tensors="pt")

# Passing Inputs to the Model

The tokenized inputs are passed to the model to generate predictions. This uses the model directly instead of relying on the pipeline abstraction.


In [17]:
from transformers import AutoModelForTokenClassification

ner_model = AutoModelForTokenClassification.from_pretrained(ner_checkpoint)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
ner_model_outputs = ner_model(**ner_model_inputs)
pprint(ner_model_outputs)

TokenClassifierOutput(
│   loss=None,
│   logits=tensor([[[ 9.4262, -2.0679, -1.3872, -1.7737, -0.8636, -1.7366, -0.9220,
│   │     -1.8739,  0.5748],
│   │    [ 9.6694, -2.4512, -0.8465, -2.9346,  0.1054, -2.0526,  0.1203,
│   │     -2.2863, -0.4128],
│   │    [ 9.8363, -2.1376, -0.6687, -2.6785,  0.0373, -2.1514, -0.3529,
│   │     -2.1566, -0.1233],
│   │    [10.2058, -2.1581, -1.0387, -2.5619, -0.2307, -1.9255, -0.1051,
│   │     -2.0321, -0.4602],
│   │    [-0.4994, -2.3452, -0.8530, -2.5723,  8.0610, -2.4264, -0.6209,
│   │     -2.7726, -0.2139],
│   │    [ 9.7179, -2.4062, -1.5056, -2.4563, -0.9515, -1.5675,  0.1270,
│   │     -1.9224, -0.2299],
│   │    [ 9.6986, -2.3140, -1.0697, -2.8253,  0.1335, -1.8696,  0.0129,
│   │     -2.0207, -0.6666],
│   │    [10.0329, -2.3599, -1.2190, -2.6853, -0.4696, -1.9200,  0.1585,
│   │     -1.9623, -0.3076],
│   │    [ 9.6650, -2.5588, -1.0664, -2.5862, -0.6822, -1.8054, -0.0527,
│   │     -1.6611,  0.1382],
│   │    [-0.4601, -2.7593, -1.5641, -2.9587, -0.0926, -2.3044,  0.8420,
│   │     -2.2897,  7.8948],
│   │    [ 9.4262, -2.0679, -1.3872, -1.7737, -0.8636, -1.7366, -0.9220,
│   │     -1.8739,  0.5748]]], grad_fn=<ViewBackward0>),
│   hidden_states=None,
│   attentions=None
)

# Interpreting Model Outputs

The raw tensor outputs are converted to probability scores over the possible entity tags for each token.


In [19]:
ner_predictions = F.softmax(ner_model_outputs.logits, dim=-1)
pprint(ner_predictions)

tensor([[[9.9972e-01, 1.0187e-05, 2.0123e-05, 1.3671e-05, 3.3970e-05,
│   │     1.4188e-05, 3.2041e-05, 1.2368e-05, 1.4314e-04],
│   │    [9.9977e-01, 5.4446e-06, 2.7094e-05, 3.3579e-06, 7.0196e-05,
│   │     8.1111e-06, 7.1250e-05, 6.4212e-06, 4.1808e-05],
│   │    [9.9981e-01, 6.3055e-06, 2.7392e-05, 3.6711e-06, 5.5495e-05,
│   │     6.2187e-06, 3.7566e-05, 6.1869e-06, 4.7261e-05],
│   │    [9.9988e-01, 4.2695e-06, 1.3077e-05, 2.8510e-06, 2.9337e-05,
│   │     5.3873e-06, 3.3263e-05, 4.8427e-06, 2.3321e-05],
│   │    [1.9138e-04, 3.0219e-05, 1.3438e-04, 2.4080e-05, 9.9915e-01,
│   │     2.7862e-05, 1.6949e-04, 1.9708e-05, 2.5460e-04],
│   │    [9.9982e-01, 5.4260e-06, 1.3354e-05, 5.1612e-06, 2.3241e-05,
│   │     1.2553e-05, 6.8334e-05, 8.8023e-06, 4.7824e-05],
│   │    [9.9979e-01, 6.0662e-06, 2.1053e-05, 3.6380e-06, 7.0118e-05,
│   │     9.4600e-06, 6.2154e-05, 8.1333e-06, 3.1503e-05],
│   │    [9.9986e-01, 4.1478e-06, 1.2980e-05, 2.9957e-06, 2.7464e-05,
│   │     6.4399e-06, 5.1468e-05, 6.1728e-06, 3.2295e-05],
│   │    [9.9978e-01, 4.9111e-06, 2.1842e-05, 4.7784e-06, 3.2076e-05,
│   │     1.0432e-05, 6.0193e-05, 1.2051e-05, 7.2856e-05],
│   │    [2.3485e-04, 2.3565e-05, 7.7863e-05, 1.9305e-05, 3.3917e-04,
│   │     3.7139e-05, 8.6359e-04, 3.7689e-05, 9.9837e-01],
│   │    [9.9972e-01, 1.0187e-05, 2.0123e-05, 1.3671e-05, 3.3970e-05,
│   │     1.4188e-05, 3.2041e-05, 1.2368e-05, 1.4314e-04]]],
│      grad_fn=<SoftmaxBackward0>)

# Converting to Human-Readable Outputs

The probabilities are parsed to extract the highest scoring entity tag per token. The start and end offsets are looked up based on the original input text.

This mirrors the output format returned by the pipeline to extract human-readable entity, score, start, end results.


In [20]:
ner_results= []
for index, prediction in enumerate(ner_predictions[0]):
  prediction_probability = torch.max(prediction).item()
  prediction_id = torch.argmax(prediction).item()
  if prediction_id > 0:
    entity = ner_model.config.id2label[prediction_id]
    word = ner_tokenizer.decode(ner_model_inputs.input_ids[0][index])
    start = ner_raw_inputs.find(word)
    end = start + len(word)
    ner_results.append({"entity":entity,"score":prediction_probability, "index": index, "word": word, "start": start, "end": end})
pprint(ner_results)

[
│   {'entity': 'I-PER', 'score': 0.9991483688354492, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19},
│   {'entity': 'I-LOC', 'score': 0.9983668923377991, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}
]

# Load QA Model

- qa_checkpoint: Specifies pretrained QA model from Hugging Face Hub to use

- pipeline: Constructs question answering pipeline object using the QA model

# Define Question and Context

- question: Question text string to ask the model

- context: Context paragraphs providing information to answer question

# Get QA Predictions

- qa_pipeline: Runs input question and context through model to make predictions

- qa_results: Contains predicted answer text and confidence score

- pprint: Prints prediction results in readable formatted output

This code loads a pretrained QA model, defines a question and context, passes them through the pipeline to generate an answer prediction, and prints the prediction nicely formatted. The pipeline handles running the inputs through the full model to output the top answer text span and score.

In [21]:
qa_checkpoint = "deepset/roberta-base-squad2"
qa_pipeline = pipeline("question-answering",model=qa_checkpoint)

question = "What is the capital of France?"
context = "The capital of France is Paris."

qa_results = qa_pipeline(question,context)
pprint(qa_results)

{'score': 0.9703434109687805, 'start': 25, 'end': 30, 'answer': 'Paris'}

Loads pretrained question answering model using AutoModelForQuestionAnswering class.

Loads corresponding tokenizer using AutoTokenizer that was used during model training.

Tokenizer preprocesses text to numeric ids.

Model generates start and end logits to predict answer span.

In [22]:
from transformers import AutoModelForQuestionAnswering

qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_checkpoint)
qa_tokenizer = AutoTokenizer.from_pretrained(qa_checkpoint)

In [23]:

qa_model_inputs = qa_tokenizer(question, context, truncation=True, padding=True, return_tensors="pt")
pprint(qa_model_inputs)

{
│   'input_ids': tensor([[   0, 2264,   16,    5,  812,    9, 1470,  116,    2,    2,  133,  812,
│   │   │   9, 1470,   16, 2201,    4,    2]]),
│   'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
}

In [24]:
qa_model_outputs = qa_model(**qa_model_inputs)
pprint(qa_model_outputs)

QuestionAnsweringModelOutput(
│   loss=None,
│   start_logits=tensor([[ 0.1126, -7.9048, -9.1849, -8.7141, -8.4898, -8.9907, -8.5684, -9.3108,
│   │    -9.2207, -8.8338, -0.7395, -2.5483, -6.9330, -3.0723, -6.1117,  4.7631,
│   │    -5.7245, -9.0346]], grad_fn=<CloneBackward0>),
│   end_logits=tensor([[ 0.4543, -7.5991, -7.6043, -8.1934, -7.4175, -7.9690, -6.5219, -6.2877,
│   │    -7.0390, -6.3119, -6.5585, -3.7544, -7.5419, -3.6280, -6.9074,  4.9452,
│   │    -0.5776, -7.0802]], grad_fn=<CloneBackward0>),
│   hidden_states=None,
│   attentions=None
)

Here is brief documentation for the provided code snippet:

# Extract Logits

- Get start and end logits from model outputs

# Get Prediction Indices

- Find index of maximum start and end logits

# Decode Answer Text

- Extract predicted answer tokens from input ids

- Convert tokens back to text with tokenizer

# Compute Probability

- Take softmax of start and end logits

- Find max joint probability of start and end

# Format Human-Readable Output

- Get start and end char offsets in context

- Format into dict with score, text, offsets

# Print Output

- Display prediction result nicely formatted

This takes the raw start and end logits from the model, picks the most likely start and end points, extracts the predicted answer text, computes the overall probability, and formats into a human-readable output with score and answer text.

In [25]:
start_logits, end_logits = qa_model_outputs.start_logits, qa_model_outputs.end_logits

In [26]:
start_pos = torch.argmax(start_logits, dim = -1)
end_pos = torch.argmax(end_logits, dim = -1) + 1
pprint((start_pos,end_pos))

(tensor([15]), tensor([16]))

In [27]:
answer_tokens = qa_model_inputs.input_ids[0][start_pos: end_pos]
answer = qa_tokenizer.decode(answer_tokens)
start_probs, end_probs = F.softmax(qa_model_outputs.start_logits,dim=-1), F.softmax(qa_model_outputs.end_logits,dim=-1)
probability = torch.max(start_probs*end_probs).item()
start = context.find(answer)
end = start + len(answer)
result = {"score": probability, "answer": answer, "start": start, "end": end}
pprint(result)

{'score': 0.9702684879302979, 'answer': ' Paris', 'start': 24, 'end': 30}